# Inventories

Inventories are a fundamental doc entrypoint for our infrastructures. 

They contain a lot of informations, including:
    
    - ansible_user
    - configuration variables in [group_name:vars]
    - host grouping eg. by geographical zones in [group_name:children]
    
    
Files:


  - [inventory](/edit/notebooks/exercise-04/inventory)
    

In [2]:
cd exercise-04

In [3]:
!cat inventory

In [6]:
# The ping module is very useful. Use it whenever you want to check connectivity!
!ansible -m ping -i inventory all[1:3]

## Inventory scripts

In [22]:
#To create custom inventory scripts just use python ;) and set it in
!grep inventory  ansible.cfg # inventory = ./docker-inventory.py

#### Exercise

in the official ansible documentation find at least 3 `ansible_connection=docker` parameters 

In [7]:
# List our containers. Note: this only works with docker-compose containers.
from __future__ import print_function
import docker
c=docker.Client(base_url="http://172.17.0.1:2375")
container_fmt = lambda x: (
    x['Names'][0][1:],
    x['Labels']['com.docker.compose.service'], 
    x['NetworkSettings']['Networks']['bridge']['IPAddress'],
)

for x in c.containers():
    print(*container_fmt(x), sep='\t\t')

In [34]:
# Ansible accepts
import json

inventories = {
    'web': {
        'hosts': ['ws-1', 'ws-2'],
    },
    'db': {
        'hosts': ['db-1', 'db-2'],
    }
}

# like this 
print(json.dumps(inventories, indent=1))
      

In [35]:
# You can pass variables to generated inventories too
inventories['web']['host_vars'] = {
    'ansible_ssh_common_args': ' -o GSSApiAuthentication=no'
}
print(json.dumps(inventories, indent=1))

#### Exercise: 

Reuse the code in [inventory-docker.py](/edit/notebooks/exercise-04/inventory-docker.py) to print a json inventory that:

  - connects via docker to "web" hosts
  - connects via ssh to "ansible" hosts 

Test it in the cell below.


In [8]:
!ansible -m ping -i inventory-docker.py all --limit ansible

#### Exercise

Modify the [inventory-docker.py](/edit/notebooks/exercise-04/inventory-docker.py) to skip StrictHostKeyChecking only on web hosts.

In [9]:
# Test here your inventory

# Configurations

You may want to split inventory files and separate prod and test environment.

## Exercise: 

split inventory in two inventory files:

  - [prod](/edit/notebooks/exercise-04/prod)  for production servers 
  - [test](/edit/notebooks/exercise-04/test)  for test servers
  
Then use `ansible -i` to explicitly use the different ones.

In [23]:
# Use this cell to test the exercise

## group_vars and host_vars

You can move variables out of inventories - eg to simplify inventory scripts - and store them in files:

 - under `group_vars` for host groups
 - under `host_vars` for single hosts

In [12]:
!tree group_vars

If you have different inventories, you can store different set of variable in custom files.
The `all` ones will be shared between all inventories

## Exercise: 
    
  - edit [group_vars/all](/edit/notebooks/exercise-04/group_vars/all) and move there all common variables from [inventory](/edit/notebooks/exercise-04/inventory) 

In [ ]:
# Test here the new inventory file

Inventory variables can store almost everything and even describe the architecture of your deployment


In [10]:
!cat group_vars/example

We can even mix and mojo `group_vars` and `inventory`, as we'll see in the next lessons.





### host_vars

Host vars can be used in automated or cloud deployments where:

  - every new host or vm, at boot, populate its own entries in `host_vars` (Eg. via file)
  - ansible is run after that setup and uses `host_vars` to configure the server and expose that values to the other machines.